In [1]:

import pandas as pd
import numpy as np
import json
import time
import random
import requests
import time
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

from IPython.display import display

pd.options.display.max_columns = None

In [2]:
pbp = pd.read_csv('docs/short-pbp-2017-18.csv')
passdf = pd.read_csv('passing-data-17-18.csv')

In [3]:
cols = passdf.columns
cols = [x.replace('\n','_').replace(' ','_') for x in cols]
passdf.columns = cols
passdf.head()

,PLAYER,TEAM,GP,W,L,MIN,PASSES_MADE,PASSES_RECEIVED,AST,SECONDARY_AST,POTENTIAL_AST,AST_ADJ,AST_TO_PASS%,AST_TO_PASS%_ADJ,Good_Passes
0,Aaron Brooks,MIN,32,19,13,189,287,335,20,1,43,21,7.0,7.3,64
1,Aaron Gordon,ORL,58,19,39,1909,2333,1884,136,13,269,155,5.8,6.6,418
2,Aaron Harrison,DAL,9,2,7,233,176,190,11,0,25,11,6.3,6.3,36
3,Abdel Nader,BOS,47,32,15,520,408,430,26,5,61,35,6.4,8.6,92
4,Adreian Payne,ORL,5,0,5,43,50,33,0,0,1,0,0.0,0.0,1


In [4]:
bp = pbp[(pbp.TO_TYPE == 'bad-pass-live') | (pbp.TO_TYPE == 'bad-pass-dead') ]
bp.head(100)

,Unnamed: 0,Unnamed: 0.1,GAME_ID,EVENTNUM,PERIOD,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,MISS_3PT_PID,MAKE_3PT_PID,MISS_2PT_PID,MAKE_2PT_PID,MISS_FT_PID,MAKE_FT_PID,AST_BY_PID,AST_TO_PID,TO_PID,TO_TYPE,STL_PID,BLK_PID,SHIFTED_PERSON1_TYPE,OREB_PID,DREB_PID,PF_PID
149,149,149,21700001,215,2,NaN,NaN,4.0,201567,Kevin Love,1.610613e+09,Cleveland,Cavaliers,CLE,5,202681,Kyrie Irving,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201567.0,bad-pass-live,202681.0,NaN,4.0,NaN,NaN,NaN
156,156,156,21700001,230,2,NaN,NaN,5.0,202681,Kyrie Irving,1.610613e+09,Boston,Celtics,BOS,4,203109,Jae Crowder,1.610613e+09,Cleveland,Cavaliers,CLE,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202681.0,bad-pass-live,203109.0,NaN,4.0,NaN,NaN,NaN
235,235,235,21700001,341,3,NaN,NaN,4.0,201565,Derrick Rose,1.610613e+09,Cleveland,Cavaliers,CLE,5,1627759,Jaylen Brown,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201565.0,bad-pass-live,1627759.0,NaN,4.0,NaN,NaN,NaN
237,237,237,21700001,347,3,NaN,NaN,4.0,2548,Dwyane Wade,1.610613e+09,Cleveland,Cavaliers,CLE,5,203935,Marcus Smart,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2548.0,bad-pass-live,203935.0,NaN,5.0,NaN,NaN,NaN
267,267,267,21700001,384,3,NaN,NaN,4.0,2548,Dwyane Wade,1.610613e+09,Cleveland,Cavaliers,CLE,5,1627759,Jaylen Brown,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2548.0,bad-pass-live,1627759.0,NaN,5.0,NaN,NaN,NaN
270,270,270,21700001,389,3,NaN,NaN,4.0,2548,Dwyane Wade,1.610613e+09,Cleveland,Cavaliers,CLE,5,202681,Kyrie Irving,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2548.0,bad-pass-live,202681.0,NaN,2.0,NaN,NaN,NaN
290,290,290,21700001,415,3,NaN,NaN,5.0,203935,Marcus Smart,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203935.0,bad-pass-dead,NaN,NaN,5.0,NaN,NaN,NaN
312,312,312,21700001,448,3,NaN,NaN,4.0,2544,LeBron James,1.610613e+09,Cleveland,Cavaliers,CLE,5,1626179,Terry Rozier,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2544.0,bad-pass-live,1626179.0,NaN,4.0,NaN,NaN,NaN
316,316,316,21700001,455,3,NaN,NaN,5.0,202681,Kyrie Irving,1.610613e+09,Boston,Celtics,BOS,4,203109,Jae Crowder,1.610613e+09,Cleveland,Cavaliers,CLE,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202681.0,bad-pass-live,203109.0,NaN,5.0,NaN,NaN,NaN
322,322,322,21700001,467,3,NaN,NaN,4.0,202684,Tristan Thompson,1.610613e+09,Cleveland,Cavaliers,CLE,5,1626179,Terry Rozier,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202684.0,bad-pass-live,1626179.0,NaN,5.0,NaN,NaN,NaN


In [5]:
plbp = bp.groupby(['PLAYER1_ID', 'PLAYER1_NAME'], as_index=False).size().reset_index(name='BAD_PASSES')

In [6]:
passdf.head()


,PLAYER,TEAM,GP,W,L,MIN,PASSES_MADE,PASSES_RECEIVED,AST,SECONDARY_AST,POTENTIAL_AST,AST_ADJ,AST_TO_PASS%,AST_TO_PASS%_ADJ,Good_Passes
0,Aaron Brooks,MIN,32,19,13,189,287,335,20,1,43,21,7.0,7.3,64
1,Aaron Gordon,ORL,58,19,39,1909,2333,1884,136,13,269,155,5.8,6.6,418
2,Aaron Harrison,DAL,9,2,7,233,176,190,11,0,25,11,6.3,6.3,36
3,Abdel Nader,BOS,47,32,15,520,408,430,26,5,61,35,6.4,8.6,92
4,Adreian Payne,ORL,5,0,5,43,50,33,0,0,1,0,0.0,0.0,1


In [7]:
df = plbp.merge(passdf,left_on='PLAYER1_NAME',right_on='PLAYER')

In [8]:
df.drop(columns=['PLAYER','TEAM','GP'],inplace=True)


In [9]:
df['Good_Pass-Bad_Pass_Ratio'] = df['Good_Passes']/df['BAD_PASSES']
df['AST-Bad_Pass_Ratio'] = df['AST']/df['BAD_PASSES']
df['AST_BP_VAL'] = df['AST-Bad_Pass_Ratio'] * df.PASSES_MADE
df['GP_BP_VAL'] = df['Good_Pass-Bad_Pass_Ratio'] * df.PASSES_MADE
df = df.round(1)
df = df[df['PASSES_MADE'] > 500]
df = df[['PLAYER1_ID','PLAYER1_NAME', 'BAD_PASSES','PASSES_MADE','AST','SECONDARY_AST','POTENTIAL_AST',
         'Good_Passes','Good_Pass-Bad_Pass_Ratio','AST-Bad_Pass_Ratio', 'AST_BP_VAL','GP_BP_VAL']]
dfastbpSort = df.sort_values('AST_BP_VAL',ascending=False).reset_index()
dfastbpSort['AST_BP_RANK'] = dfastbpSort.index + 1 
dfastbpSort.head(10)
df2 = dfastbpSort.iloc[:11,1:11]
df2

,PLAYER1_ID,PLAYER1_NAME,BAD_PASSES,PASSES_MADE,AST,SECONDARY_AST,POTENTIAL_AST,Good_Passes,Good_Pass-Bad_Pass_Ratio,AST-Bad_Pass_Ratio
0,203915,Spencer Dinwiddie,77,4302,508,39,974,1521,19.8,6.6
1,201954,Darren Collison,49,3421,368,34,700,1102,22.5,7.5
2,202397,Ish Smith,56,3995,360,50,752,1162,20.8,6.4
3,203999,Nikola Jokic,96,4975,458,40,703,1201,12.5,4.8
4,1627732,Ben Simmons,167,5929,658,82,1371,2111,12.6,3.9
5,1626179,Terry Rozier,35,3416,228,44,463,735,21.0,6.5
6,203507,Giannis Antetokounmpo,79,4246,354,59,689,1102,13.9,4.5
7,200768,Kyle Lowry,127,4484,537,41,951,1529,12.0,4.2
8,101108,Chris Paul,72,2926,457,35,895,1387,19.3,6.3
9,1626145,Tyus Jones,33,2611,232,24,414,670,20.3,7.0


In [10]:
dfgpbpSort = df.sort_values('GP_BP_VAL',ascending=False).reset_index()
# dfgpbpSort.drop(columns=[''])
dfgpbpSort['GP_BP_RANK'] = dfgpbpSort.index + 1 
dfgpbpSort.head(10)

,index,PLAYER1_ID,PLAYER1_NAME,BAD_PASSES,PASSES_MADE,AST,SECONDARY_AST,POTENTIAL_AST,Good_Passes,Good_Pass-Bad_Pass_Ratio,AST-Bad_Pass_Ratio,AST_BP_VAL,GP_BP_VAL,GP_BP_RANK
0,261,203915,Spencer Dinwiddie,77,4302,508,39,974,1521,19.8,6.6,28382.0,84978.5,1
1,144,202397,Ish Smith,56,3995,360,50,752,1162,20.8,6.4,25682.1,82896.2,2
2,108,201954,Darren Collison,49,3421,368,34,700,1102,22.5,7.5,25692.4,76937.6,3
3,360,1627732,Ben Simmons,167,5929,658,82,1371,2111,12.6,3.9,23361.0,74946.8,4
4,336,1626179,Terry Rozier,35,3416,228,44,463,735,21.0,6.5,22252.8,71736.0,5
5,293,203999,Nikola Jokic,96,4975,458,40,703,1201,12.5,4.8,23734.9,62239.3,6
6,239,203507,Giannis Antetokounmpo,79,4246,354,59,689,1102,13.9,4.5,19026.4,59229.0,7
7,96,201609,Goran Dragic,86,4268,356,64,744,1164,13.5,4.1,17667.5,57766.9,8
8,33,101108,Chris Paul,72,2926,457,35,895,1387,19.3,6.3,18572.0,56366.1,9
9,329,1626170,Jerian Grant,56,2999,342,32,661,1035,18.5,6.1,18315.3,55427.9,10
